## Image Stiching

In [2]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib 

# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV 
import cv2 as cv
print(f'Version de OpenCV: {cv.__version__}')
# Usamos las poderosas herramientas de graficación de matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg
Version de OpenCV: 3.4.2


In [3]:
# Cargamos la imagen a procesar
img_left = cv.imread('imagenes/pano_1.jpg',cv.COLOR_BGR2GRAY)
img_right = cv.imread('imagenes/pano_2.jpg')

# La transformamos en escala de grises
gray_left = cv.cvtColor(img_left, cv.COLOR_BGR2GRAY)
gray_right = cv.cvtColor(img_right, cv.COLOR_BGR2GRAY)
img_left = cv.cvtColor(img_left, cv.COLOR_BGR2RGB)
img_right = cv.cvtColor(img_right, cv.COLOR_BGR2RGB)


plt.subplot(1,2,1)
plt.imshow(img_left)
plt.subplot(1,2,2)
plt.imshow(img_right)

#### 1. Obtenemos los descriptores

In [4]:
# Creamos el vector de características SIFT
sift = cv.xfeatures2d.SIFT_create()

# Y buscamos según el algoritmo...
kps_left, descriptors_left = sift.detectAndCompute(gray_left, None)
kps_right, descriptors_right = sift.detectAndCompute(gray_right, None)

# me quedo solo con valores x,y de la posicion de los descriptores
kps_loc_left = np.float32([kp.pt for kp in kps_left])
kps_loc_right = np.float32([kp.pt for kp in kps_right])

# BFMatcher con parámetros por defecto
bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)
matches = bf.match(descriptors_left, descriptors_right)

# Los ordenamos según distancia
matches = sorted(matches, key = lambda x:x.distance)

# Dibujamos las primeras 10 coincidencias
n=10
img_matched = cv.drawMatches(img_left,kps_left,img_right,kps_right,matches[:n],None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.imshow(img_matched)

In [5]:
if len(matches) > 4:
    # construct the two sets of points
    ptsA = np.float32([kps_loc_left[m.queryIdx] for m in matches])
    ptsB = np.float32([kps_loc_right[m.trainIdx] for m in matches])
    
    # Calculamos la matriz de homografia
    (H, status) = cv.findHomography(ptsB, ptsA, cv.RANSAC, 4.0)
    
    result = cv.warpPerspective(img_right, H,(img_left.shape[1] + img_right.shape[1], img_right.shape[0]))
    result[0:img_left.shape[0], 0:img_right.shape[1]] = img_left
    # mostramos la imagen
    plt.figure()
    plt.imshow(result[100:-100,:-100,:])